# Master notebook 6/2024

In [1]:
from __future__ import division, unicode_literals, print_function  # for compatibility with Python 2 and 3
import batch_roi_selector

import matplotlib as mpl
import matplotlib.pyplot as plt


import numpy as np
import pandas as pd
from pandas import DataFrame, Series  # for convenience

import pims
import trackpy as tp
import os
import glob
import nd2
import seaborn as sns

In [2]:
nd2_directory = 'C:/Users/Haku/DATA/nd2test/'

tif_directory = 'C:/Users/Haku/DATA/something/'

ndsfiles = 'C:/Users/Haku/DATA/MASTERTEST/'
master = 'C:/Users/Haku/DATA/MASTERTEST_output/'

OPTIONAL: crop ND2 images and make into tiffs

In [3]:
import batch_roi_selector

# Set your input and output directories


# Set the ROI size
roi_width = 100
roi_height = 100

# Run the function to process the directory
# batch_roi_selector.process_directory(nd2_directory, tif_directory, roi_width, roi_height) #
batch_roi_selector.process_directory(ndsfiles, master, roi_width, roi_height) #


ROI center set to: (256, 256), size: (100, 100) for file: Zeus_banana.nd2
Metadata saved to CSV.
Metadata saved to CSV.
All conditions processed.


Main pytrack pipeline:

In [ ]:
# change the following to %matplotlib notebook for interactive plotting
%matplotlib inline

# Optionally, tweak styles.
mpl.rc('figure',  figsize=(10, 5))
mpl.rc('image', cmap='gray')

In [ ]:
# Use glob to find TIF files in the directory
tif_files = glob.glob(tif_directory + '*.tif')
# Print the list of TIF files
print(tif_files)

In [ ]:
####### detection settings ########
detection_circle_size = 9
minimum_intensity = 150
iterations = 50
svn_threshold = 10

####### linking settings ########
linking_max_distance = 7
disappearance_max_frames = 2

####### filtering settings ########
minimum_track_length = 10
size = 2

# Create an empty master dataframe
master_df = pd.DataFrame()

tp.quiet()  # Turn off progress reports for best performance
for i in range(len(tif_files)):
    frames = pims.open(tif_files[i])
    f = tp.batch(frames[:], detection_circle_size, minmass=minimum_intensity, invert=False, max_iterations=iterations, threshold=svn_threshold)
    t = tp.link(f, linking_max_distance, memory=disappearance_max_frames)
    t1 = tp.filter_stubs(t, minimum_track_length)
    t2 = t1[((t1['mass'] > minimum_intensity) & (t1['size'] < size))]
    d = tp.compute_drift(t2)
    tm = tp.subtract_drift(t2.copy(), d)
    # Add column with the filename
    tm['filename'] = os.path.splitext(os.path.basename(tif_files[i]))[0]
    # Append tm dataframe to the master dataframe
    master_df = pd.concat([master_df, tm])

    # tm.to_csv(tif_files[i] + '.csv')
    print(tif_files[i] + ' processed')
    print('-------------------------------------')
    # reset index
master_df = master_df.reset_index(drop=True)
# create a file ID (integer) based on each filename
master_df['file_id'] = pd.Categorical(master_df['filename']).codes
# # create a uniq_id for each particle, based on the filename and the particle number
master_df['unique_id'] = master_df['file_id'].astype(str) + '_' + master_df['particle'].astype(str)

master_df.to_csv(tif_directory + 'master_df.csv')

In [ ]:
# load csv master dataframe
master_df = pd.read_csv(tif_directory + 'master_df.csv')

In [ ]:
master_df

In [ ]:
# select a random number from the unique particles
random_particle = np.random.choice(master_df['particle'].unique())

# extract a new 'particle df' for a random particle


particle_df = master_df[master_df['particle'] == random_particle]
particle_df

In [ ]:

master_df


In [ ]:
# create a file ID (integer) based on each filename
master_df['file_id'] = pd.Categorical(master_df['filename']).codes
# # create a uniq_id for each particle, based on the filename and the particle number
master_df['unique_id'] = master_df['file_id'].astype(str) + '_' + master_df['particle'].astype(str)

In [ ]:
master_df

In [ ]:
# Count the number of frames for each particle
frame_counts = master_df['particle'].value_counts()

# Compute the mean track length across all particles
mean_track_length_value_counts = frame_counts.mean()
median_track_length_value_counts = frame_counts.median()

print('Mean track length: ' + str(mean_track_length_value_counts))
print('Median track length: ' + str(median_track_length_value_counts))


In [ ]:

sns.histplot(master_df['unique_id'].value_counts(), kde=False)
plt.xlabel('Track length (frames)')
plt.ylabel('Count')

# Annotate plot with mean and median
mean = master_df['particle'].value_counts().mean()
median = master_df['particle'].value_counts().median()
plt.text(0.5, 0.9, f"Mean: {mean:.2f} frames", transform=plt.gca().transAxes)
plt.text(0.5, 0.85, f"Median: {median:.2f} frames", transform=plt.gca().transAxes)

plt.show()
plt.show()


In [ ]:
# value counts of the unique_id
valuecounts = master_df['unique_id'].value_counts()
# valuecounts
# translate each value count to time in seconds by multiplying by 0.1
valuecounts_seconds = valuecounts * 0.1
valuecounts_seconds

sns.histplot(valuecounts_seconds, kde=False)
plt.xlabel('Track length (seconds)')
plt.ylabel('Count')

# Annotate plot with mean and median
mean = valuecounts_seconds.mean()
median = valuecounts_seconds.median()
plt.text(0.5, 0.9, f"Mean: {mean:.2f} seconds", transform=plt.gca().transAxes)
plt.text(0.5, 0.85, f"Median: {median:.2f} seconds", transform=plt.gca().transAxes)

plt.show()
plt.show()


In [ ]:
# make a new df, where each particle has a single row, and the only other two columns are the max time [s] and frame for that particle
max_time = master_df.groupby('particle')['frame'].max()
# max_frame = master_df.groupby('particle')['frame'].max()
max_time_df = pd.DataFrame(max_time)
max_time_df = max_time_df.reset_index()
max_time_df.columns = ['particle', 'max_frame']
max_time_df['max_time'] = max_time_df['max_frame'] * 0.1
max_time_df


In [ ]:
# make a seaborn histogram from that
sns.histplot(max_time_df['max_time'], kde=False, bins = 100)

In [ ]:
plt.figure(figsize=(6, 6))  # Set the figure size to create a square plot
tp.plot_traj(tm)
plt.axis('equal')  # Set the aspect ratio to be equal
plt.show()

In [ ]:
micronsperpixel=0.107
framespersecond = 10
timebetweenframes = 1/framespersecond
print(timebetweenframes)

average_track_length = tm['particle'].value_counts().mean()
average_track_length2 = tm['particle'].value_counts().median()

print(average_track_length)
print(average_track_length2)

In [ ]:
micronsperpixel=0.107
framespersecond = 10
timebetweenframes = 1/framespersecond
# print(timebetweenframes)
# add a column with the time in seconds to the dataframe
master_df['time [s]'] = master_df['frame'] * timebetweenframes

In [ ]:
framespersecond = 10
tracklengthsinseconds = tm['particle'].value_counts() / framespersecond
# add time column

# print(tracklengthsinseconds)

sns.histplot(tracklengthsinseconds, kde=False, bins=100, )
plt.xlabel('Track Length (seconds)')
plt.ylabel('Count')
plt.title('Histogram of Track Length')
plt.show()

In [ ]:
# 

In [ ]:
# extract a single particle from the dataframe
particle = 1
single_particle = master_df[master_df['particle'] == particle]
single_particle.head()

In [ ]:
master_df['time [s]'] = master_df['frame'] * timebetweenframes

In [ ]:
master_df

In [ ]:
max_time_per_particle = master_df.groupby('particle')['time [s]'].max()
max_time_per_particle.mean()

In [ ]:
# histogram of max_time_per_particle
sns.histplot(max_time_per_particle, kde=False, bins=100)

Movie function. Takes the current tracks_df and the full movie path you want to convert

In [ ]:
tracks_df = tm

In [ ]:
tracks_df.reset_index(drop=True, inplace=True)

In [ ]:
movie_path = tif_files[1]
print(movie_path)

In [ ]:
import os
import pims

import matplotlib.pyplot as plt

def overlay_tracks_with_movie(tracks_df, movie_path):
    # Load the raw movie
    frames = pims.open(movie_path)
    
    # Create a new folder to save the PNG images
    output_folder = os.path.splitext(movie_path)[0]
    os.makedirs(output_folder, exist_ok=True)
    
    # Iterate over each frame in the movie
    for frame_index, frame in enumerate(frames):
        # Create a figure and axis
        fig, ax = plt.subplots()
        
        # Iterate over each track in the DataFrame
        for particle_id, track in tracks_df.groupby('particle'):
            # Get the x and y coordinates of the track for the current frame
            x = track.loc[track['frame'] <= frame_index, 'x']
            y = track.loc[track['frame'] <= frame_index, 'y']
            
            # Plot the track as a line
            ax.plot(x, y, label=f'Track {particle_id}')
        
        # Display the current frame
        ax.imshow(frame, cmap='gray')
        
        # Set the axis labels
        ax.set_xlabel('x [px]')
        ax.set_ylabel('y [px]')
        
        # Add a legend
        # ax.legend()
        
        # Save the figure as a PNG image in the output folder
        output_path = os.path.join(output_folder, f'frame_{frame_index}.png')
        plt.savefig(output_path)
        
        # Close the figure to free up memory
        plt.close(fig)

In [ ]:
overlay_tracks_with_movie(tracks_df, movie_path)